In [ ]:
import numpy as np
import re
import nltk
from datasets import load_dataset
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F

from sklearn.metrics import accuracy_score
from sklearn.model_selection import ParameterGrid

import matplotlib.pyplot as plt


import pytorch_lightning as pl
from torchmetrics import Accuracy


/opt/anaconda3/envs/sc4002/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /Users/jz/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /Users/jz/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /Users/jz/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /Users/jz/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /Users/jz/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged

True

# Part 0. Dataset Preparation

In [3]:
# loading the dataset from the library
dataset = load_dataset("rotten_tomatoes")
train_dataset = dataset ['train']
validation_dataset = dataset ['validation']
test_dataset = dataset ['test']

In [4]:
# check the sizes of each dataset
train_size = len(train_dataset)
validation_size = len(validation_dataset)
test_size = len(test_dataset)

print(f"Training dataset size: {train_size}")
print(f"Validation dataset size: {validation_size}")
print(f"Test dataset size: {test_size}")

Training dataset size: 8530
Validation dataset size: 1066
Test dataset size: 1066


In [5]:
# view an example from each dataset
print("Train Dataset")
print(train_dataset.features)
print(train_dataset[0]) 

print("Test Dataset")
print(test_dataset.features)
print(test_dataset[0]) 

print("Validation Dataset")
print(validation_dataset.features)
print(validation_dataset[0])

Train Dataset
{'text': Value(dtype='string', id=None), 'label': ClassLabel(names=['neg', 'pos'], id=None)}
{'text': 'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .', 'label': 1}
Test Dataset
{'text': Value(dtype='string', id=None), 'label': ClassLabel(names=['neg', 'pos'], id=None)}
{'text': 'lovingly photographed in the manner of a golden book sprung to life , stuart little 2 manages sweetness largely without stickiness .', 'label': 1}
Validation Dataset
{'text': Value(dtype='string', id=None), 'label': ClassLabel(names=['neg', 'pos'], id=None)}
{'text': 'compassionately explores the seemingly irreconcilable situation between conservative christian parents and their estranged gay and lesbian children .', 'label': 1}


# Part 1. Preparing Word Embeddings

### Preprocessing

In [6]:
def preprocessing(text):

    # remove any other special characters but keep the general ones for potential sentiment usage
    text = re.sub(r'[^a-zA-Z0-9\'\!\?\.]', ' ', text)
    
    # replace multiple spaces with one space only
    text = re.sub(r'\s+', ' ', text)

    # remove leading and trailing whitespace to avoid unnecessary inconsistency 
    text = text.strip()

    return text

# apply the preprocessing function to the 'text' column of each dataset
train_dataset = train_dataset.map(lambda x: {'text': preprocessing(x['text'])})
validation_dataset = validation_dataset.map(lambda x: {'text': preprocessing(x['text'])})
test_dataset = test_dataset.map(lambda x: {'text': preprocessing(x['text'])})

# an example of the processed text
print("Train Dataset Example:")
print(train_dataset[0])

Train Dataset Example:
{'text': "the rock is destined to be the 21st century's new conan and that he's going to make a splash even greater than arnold schwarzenegger jean claud van damme or steven segal .", 'label': 1}


In [7]:
# tokenization
# empty list to store the resulting sentences
tokenized_sentences = []

for text in train_dataset['text']:
    # Tokenize the text and append the tokenized sentence to the list
    tokenized_sentences.append(word_tokenize(text))

### (a) Size of vocabulary in training data

In [8]:
# empty set for storing unique words
original_vocab = set()

for sentence in tokenized_sentences:
    for word in sentence:
        # add each word in the sentence to the words set
        original_vocab.add(word)

print(f"(a) The size of vocabulary formed in the training data is {len(original_vocab)}")

(a) The size of vocabulary formed in the training data is 16683


### (b) Number of OOV in the training data

In [9]:
# adjust the parameters for word2vec
vector_size = 100 # Dimensionality of the word vectors
window = 3 # Maximum distance between the current and predicted word within a sentence
min_count = 2 # Ignores all words with total frequency lower than this
workers = 4 # CPU cores
sg = 1 # 1 for skip-gram, 0 for CBOW
epochs = 5 

# train the word2vec model
word2vec_model = Word2Vec(
    sentences = tokenized_sentences, 
    vector_size = vector_size, 
    window = window, 
    min_count = min_count, 
    workers = workers,
    epochs = epochs)

# variable to store model's vocab list 
word2vec_vocab = set(word2vec_model.wv.key_to_index)

# Calculate OOV words by comparing the original vocab and Word2Vec vocab
oov_words = original_vocab - word2vec_vocab

print(f"(b) Number of OOV words in the training data is {len(oov_words)} when the minimum threshold for each word is {min_count}")

(b) Number of OOV words in the training data is 7866 when the minimum threshold for each word is 2


### (c) Mitigating OOV

In [10]:
# Replace all OOV words with <UNK>

# define the UNK and PAD token
UNK_TOKEN = '<UNK>'
PAD_TOKEN = '<PAD>'

# process each sentence in the tokenized_sentences list
for i, sentence in enumerate(tokenized_sentences):
    # empty list to store the current processed sentence
    processed_sentence = []
    for word in sentence:
        if word in word2vec_vocab:
            # if the current word is in the model's vocab, keep it as it is
            processed_sentence.append(word)  
        else:
            # otherwise, replace the word with UNK
            processed_sentence.append(UNK_TOKEN) 

    # update the sentence in the original tokenized_sentences list
    tokenized_sentences[i] = processed_sentence

### Embedding matrix

In [11]:
# empty set for storing unique words
final_vocab = set()

for sentence in tokenized_sentences:
    for word in sentence:
        # add each word in the sentence to the final_vocab set
        final_vocab.add(word)

# add 'UNK' and '<PAD>' to the vocabulary
final_vocab.add(UNK_TOKEN)
final_vocab.add(PAD_TOKEN)

# create the dictionary that maps each word in final_vocab to a unique index
word_to_index = {word: i for i, word in enumerate(final_vocab)}

embedding_dim = word2vec_model.vector_size 

# initialize embedding matrix with number of vocab and embedding dimension
embedding_matrix = np.zeros((len(word_to_index), embedding_dim))

# fill the embedding matrix with the corresponding word vectors
for word, i in word_to_index.items():
    if word in word2vec_model.wv:
        embedding_matrix[i] = word2vec_model.wv[word]
    elif word == PAD_TOKEN:
        # give padding token a zero vector to have no impact on the word semantics
        embedding_matrix[i] = np.zeros(embedding_dim)
    else:
        # use average vector for unknown words 
        embedding_matrix[i] = np.mean(word2vec_model.wv.vectors, axis=0)

print(f"Shape of embedding matrix: {embedding_matrix.shape}")

Shape of embedding matrix: (8819, 100)


In [12]:
# convert word to indices 
def words_to_indices(sentence, word_to_index):
    return [word_to_index.get(word, word_to_index[UNK_TOKEN]) for word in sentence.split()]

train_X = [words_to_indices(sentence, word_to_index) for sentence in train_dataset['text']]
train_y = train_dataset['label']
val_X = [words_to_indices(sentence, word_to_index) for sentence in validation_dataset['text']]
val_y = validation_dataset['label']
test_X = [words_to_indices(sentence, word_to_index) for sentence in test_dataset['text']]
test_y = test_dataset['label']

def create_dataloader(X, y, batch_size=16, shuffle=True):
    X_tensor = [torch.tensor(seq, dtype=torch.long) for seq in X]
    X_padded = pad_sequence(X_tensor, batch_first=True, padding_value=word_to_index[PAD_TOKEN])
    y_tensor = torch.tensor(y, dtype=torch.long)
    dataset = TensorDataset(X_padded, y_tensor)
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

train_dataloader = create_dataloader(train_X, train_y, shuffle=True)
val_dataloader = create_dataloader(val_X, val_y, shuffle=False)
test_dataloader = create_dataloader(test_X, test_y, shuffle=False)

# convert embedding_matrix to tensor
embedding_matrix = torch.FloatTensor(embedding_matrix)

# Part 3.3 Bi-LSTM and Bi-GRU

In [ ]:

class SentimentBiLSTM(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, output_dim, pad_idx, embedding_matrix, 
                 freeze_embeddings=True, dropout_rate=0.5, num_layers=1):
        super(SentimentBiLSTM, self).__init__()
        
        embedding_tensor = torch.FloatTensor(embedding_matrix)
        
        self.embedding = nn.Embedding.from_pretrained(embedding_tensor, padding_idx=pad_idx, 
                                                      freeze=freeze_embeddings)
        
        self.lstm = nn.LSTM(input_size=embedding_dim,
                            hidden_size=hidden_dim,
                            num_layers=num_layers,
                            bidirectional=True, 
                            batch_first=True)
        
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(dropout_rate)
        

    def forward(self, text):
      
        embedded = self.dropout(self.embedding(text))
        
        lstm_output, (hidden, cell) = self.lstm(embedded)
        
        hidden_cat = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)
        hidden_cat = self.dropout(hidden_cat)
        
        output = self.fc(hidden_cat)
        
        return output
    

class SentimentBiGRU(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, output_dim, pad_idx, embedding_matrix, 
                 freeze_embeddings=True, dropout_rate=0.5, num_layers=1):
        super(SentimentBiGRU, self).__init__()
        
        embedding_tensor = torch.FloatTensor(embedding_matrix)
        
        self.embedding = nn.Embedding.from_pretrained(embedding_tensor, padding_idx=pad_idx, 
                                                      freeze=freeze_embeddings)
        
        self.gru = nn.GRU(input_size=embedding_dim,
                            hidden_size=hidden_dim,
                            num_layers=num_layers,
                            bidirectional=True, 
                            batch_first=True)
        
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(dropout_rate)
        

    def forward(self, text):
       
        embedded = self.dropout(self.embedding(text))
        
        gru_output, hidden = self.gru(embedded)

        hidden_cat = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)
        hidden_cat = self.dropout(hidden_cat)

        output = self.fc(hidden_cat)
        
        return output


In [14]:
class SentimentClassifier(pl.LightningModule):
    def __init__(self, model, learning_rate=1e-3):
        super(SentimentClassifier, self).__init__()
        self.model = model
        self.criterion = torch.nn.CrossEntropyLoss()
        self.learning_rate = learning_rate
        
        # Initialize accuracy metrics
        self.train_acc = Accuracy(task="multiclass", num_classes=2)
        self.val_acc = Accuracy(task="multiclass", num_classes=2)
        self.test_acc = Accuracy(task="multiclass", num_classes=2)
        
        # For storing epoch loss
        self.train_losses = []
        self.val_losses = []
        self.test_losses = []

    def forward(self, text):
        return self.model(text)

    def training_step(self, batch, batch_idx):
        text, labels = batch
        outputs = self(text)
        loss = self.criterion(outputs, labels)
        acc = self.train_acc(outputs, labels)
        
        # Store loss for logging at epoch end
        self.train_losses.append(loss.item())
        
        self.log('train_loss', loss, prog_bar=True)
        self.log('train_acc', acc, prog_bar=True)
        
        return loss

    def on_train_epoch_end(self):
        avg_train_loss = sum(self.train_losses) / len(self.train_losses)
        avg_train_acc = self.train_acc.compute()
        
        print(f"Train Loss: {avg_train_loss:.4f}, Train Acc: {avg_train_acc:.4f}")
        
        # Clear accumulated values
        self.train_losses.clear()
        self.train_acc.reset()

    def validation_step(self, batch, batch_idx):
        text, labels = batch
        outputs = self(text)
        loss = self.criterion(outputs, labels)
        acc = self.val_acc(outputs, labels)
        
        # Store loss for logging at epoch end
        self.val_losses.append(loss.item())
        
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', acc, prog_bar=True)
        
        return loss

    def on_validation_epoch_end(self):
        avg_val_loss = sum(self.val_losses) / len(self.val_losses)
        avg_val_acc = self.val_acc.compute()
        
        print(f"Val Loss: {avg_val_loss:.4f}, Val Acc: {avg_val_acc:.4f}")
        
        # Clear accumulated values
        self.val_losses.clear()
        self.val_acc.reset()

    # New test_step method for testing phase
    def test_step(self, batch, batch_idx):
        text, labels = batch
        outputs = self(text)
        loss = self.criterion(outputs, labels)
        acc = self.test_acc(outputs, labels)
        
        # Store loss for logging at epoch end
        self.test_losses.append(loss.item())
        
        self.log('test_loss', loss, prog_bar=True)
        self.log('test_acc', acc, prog_bar=True)
        
        return loss

    def on_test_epoch_end(self):
        avg_test_loss = sum(self.test_losses) / len(self.test_losses)
        avg_test_acc = self.test_acc.compute()
        
        print(f"Test Loss: {avg_test_loss:.4f}, Test Acc: {avg_test_acc:.4f}")
        
        # Clear accumulated values
        self.test_losses.clear()
        self.test_acc.reset()

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.learning_rate)

In [16]:


embedding_dim = embedding_matrix.shape[1]  # match word2vec vector size
hidden_dim = 280
output_dim = 2  
dropout_rate = 0.2684676960025334
num_layers = 2
learning_rate = 0.00024410733132567515


# Create the model instance (biLSTM and biGRU)
bilstm_model = SentimentBiLSTM(
                                embedding_dim=embedding_dim,
                                hidden_dim=hidden_dim,
                                output_dim=output_dim,
                                pad_idx=word_to_index[PAD_TOKEN], 
                                embedding_matrix=embedding_matrix,
                                freeze_embeddings=False, 
                                dropout_rate=dropout_rate,
                                num_layers=num_layers
                               )



In [17]:

# BiLSTM Training and Evaluation

bilstm_classifier = SentimentClassifier(model=bilstm_model, learning_rate=learning_rate)

early_stopping = pl.callbacks.EarlyStopping(monitor='val_loss', patience=5, mode='min')
model_checkpoint = pl.callbacks.ModelCheckpoint(monitor='val_loss', mode='min', save_top_k=1)

trainer = pl.Trainer(max_epochs=30, callbacks=[early_stopping, model_checkpoint])

# Training
trainer.fit(bilstm_classifier, train_dataloader, val_dataloader)

# Evaluation
trainer.test(bilstm_classifier, test_dataloader)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/anaconda3/envs/sc4002/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default

  | Name      | Type               | Params | Mode 
---------------------------------------------------------
0 | model     | SentimentBiLSTM    | 3.6 M  | train
1 | criterion | CrossEntropyLoss   | 0      | train
2 | train_acc | MulticlassAccuracy | 0      | train
3 | val_acc   | MulticlassAccuracy | 0      | train
4 | test_acc  | Multiclass

Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/anaconda3/envs/sc4002/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Sanity Checking DataLoader 0: 100%|██████████| 2/2 [00:00<00:00,  2.25it/s]Val Loss: 0.6827, Val Acc: 1.0000
                                                                           

/opt/anaconda3/envs/sc4002/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Epoch 7: 100%|██████████| 534/534 [00:18<00:00, 29.21it/s, v_num=5, train_loss=0.00593, train_acc=1.000, val_loss=1.170, val_acc=0.744]


/opt/anaconda3/envs/sc4002/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 67/67 [00:00<00:00, 75.63it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.7514071464538574
        test_loss           1.0231971740722656
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 1.0231971740722656, 'test_acc': 0.7514071464538574}]

In [18]:
# BiGRU Training and Evaluation
embedding_dim = embedding_matrix.shape[1]  # match word2vec vector size
hidden_dim = 499
output_dim = 2  
dropout_rate = 0.2684676960025334
num_layers = 4
learning_rate = 0.00012151617026673395


bigr_model = SentimentBiGRU(
                                embedding_dim=embedding_dim,
                                hidden_dim=hidden_dim,
                                output_dim=output_dim,
                                pad_idx=word_to_index[PAD_TOKEN], 
                                embedding_matrix=embedding_matrix,
                                freeze_embeddings=False,
                                dropout_rate=dropout_rate,
                                num_layers=num_layers)

# Print the model architecture
print(bilstm_model)
print(bigr_model)

bigr_classifier = SentimentClassifier(model=bigr_model)

early_stopping = pl.callbacks.EarlyStopping(monitor='val_loss', patience=5, mode='min')
model_checkpoint = pl.callbacks.ModelCheckpoint(monitor='val_loss', mode='min', save_top_k=1)

trainer = pl.Trainer(max_epochs=30, callbacks=[early_stopping, model_checkpoint])

# Training
trainer.fit(bigr_classifier, train_dataloader, val_dataloader)

# Evaluation
trainer.test(bigr_classifier, test_dataloader)

SentimentBiLSTM(
  (embedding): Embedding(8819, 100, padding_idx=5206)
  (lstm): LSTM(100, 280, num_layers=2, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=560, out_features=2, bias=True)
  (dropout): Dropout(p=0.2684676960025334, inplace=False)
)
SentimentBiGRU(
  (embedding): Embedding(8819, 100, padding_idx=5206)
  (gru): GRU(100, 499, num_layers=4, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=998, out_features=2, bias=True)
  (dropout): Dropout(p=0.2684676960025334, inplace=False)
)


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name      | Type               | Params | Mode 
---------------------------------------------------------
0 | model     | SentimentBiGRU     | 16.1 M | train
1 | criterion | CrossEntropyLoss   | 0      | train
2 | train_acc | MulticlassAccuracy | 0      | train
3 | val_acc   | MulticlassAccuracy | 0      | train
4 | test_acc  | MulticlassAccuracy | 0      | train
---------------------------------------------------------
16.1 M    Trainable params
0         Non-trainable params
16.1 M    Total params
64.589    Total estimated model params size (MB)
9         Modules in train mode
0         Modules in eval mode


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/anaconda3/envs/sc4002/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Sanity Checking DataLoader 0: 100%|██████████| 2/2 [00:00<00:00,  2.95it/s]Val Loss: 0.7335, Val Acc: 0.0000
                                                                           

/opt/anaconda3/envs/sc4002/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Epoch 6: 100%|██████████| 534/534 [05:28<00:00,  1.63it/s, v_num=6, train_loss=0.000791, train_acc=1.000, val_loss=1.630, val_acc=0.711]


/opt/anaconda3/envs/sc4002/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 67/67 [00:09<00:00,  6.84it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.7232645153999329
        test_loss           1.4577481746673584
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 1.4577481746673584, 'test_acc': 0.7232645153999329}]